In [ ]:
import traceback
from utils.getDataFiles import *
from utils.alignmentFunctions import get_analog_times, align_scope_triggers_to_frames
from utils.readIntan import *
from getFacemapData import *

class TwoPData:
    '''
    Class to encapsulate ALL recording data 

    Args:
        data_basepath (str or Path-like): filepath where suite2p and intan data are located 

        twop_channel (int; default=2): channel of recorded scope

        pd_channel (int; default=5): channel of photodiode

        camera_channel (int; default=3): channel of camera

        treadmill_channel (int; default=6): channel of treadmill
    '''
    def __init__(self, data_basepath, twop_channel=2, pd_channel=5, camera_channel=3, treadmill_channel=6):
        self.data_basepath = data_basepath
        self.twop_chan = twop_channel
        self.pd_chan = pd_channel
        self.camera_chan = camera_channel
        self.treadmill_chan = treadmill_channel
        # get all intan and s2p data
        (self.s2p_out, self.fs_intan, self.photodiode_raw,
          self.twop_raw, self.camera_raw, self.treadmill_raw) = get_all_recording_data(data_basepath,
                                                                                       twop_chan=twop_channel, pd_chan=pd_channel,
                                                                                       camera_chan=camera_channel, treadmill_chan=treadmill_channel)  
        ## get times in seconds of TTL triggers for scope, photodiode, camera, and treadmill
        self.scope_times, self.scope_times_end = get_analog_times(self.twop_raw, upTransition=True)
        try:
            self.scope_times, self.scope_times_end = align_scope_triggers_to_frames(self.s2p_out, self.scope_times)
        except:
            print(f'Could not confirm alignment of scope and triggers.')
        if self.photodiode_raw is not None:
            self.pd_times, self.pd_times_end = get_analog_times(self.photodiode_raw)
        if self.camera_chan is not None:
            self.camera_times, self.camera_times_end = get_analog_times(self.camera_raw)
        if treadmill_channel is not None:
            self.treadmill_times, self.treadmill_times_end = get_analog_times(self.treadmill_raw) 
        # read in facemap data
        self.facemap_data = get_facemap_data(self.data_basepath)

    def make_frame_df(self, output_csv=True, output_filepath=None):
        """
        Create a dataframe of relative time estimates from trigger and raw frame times in seconds since recording start

        Args:
            output_csv (bool): whether to output a csv  
            output_filepath (str, Path-like):
        Returns:
            scope_df (pd.DataFrame): dataframe of raw scope times 
        """
        timeEst = np.arange(self.s2p_out.nframes) / self.s2p_out.scope_fs
        scope_df = pd.DataFrame({'timeEst':timeEst, 'frame_time':self.scope_times})
        if output_csv:
            if output_filepath is None:
                output_filepath = os.path.join(self.data_basepath, 'frame_times.csv')
            # if file ending doesnt end with csv add extension
            if not output_filepath.endswith('.csv'):
                output_filepath += '.csv'
            scope_df.to_csv(output_filepath)
        return scope_df
    
    def make_state_df(self, cam_fps=30, treadmill_data=True, smoothing_kernel=5, movement_percentile=70,
                       output_csv=False, output_filepath=None):
        """
        Create state dataframe (from getFacemapData.py) using data from base folder

        Args:
            cam_fps (int; default is 30): camera ttl rate

            treadmill_data (bool; default is True): whether treadmill data was recorded

            smoothing_kernel (int): smoothing factor for treadmill signal

            movement_percentile (int; default=70): percentile for movement detection

            output_csv (bool): whether to output a csv file, if so provide a path to the output_filepath argument 

            output_filepath (str, Path-like): path to save CSV file to if output_csv == True

        Returns:
            state_dataframe (pd.DataFrame): dataframe containing camera aligned timestamps, treadmill, motion, and pupil signals
        """
        state_dataframe = get_state_df(facemap_data= self.facemap_data,
                                camera_times= self.camera_times,
                                treadmill_data=treadmill_data, treadmill_signal= self.treadmill_raw,
                                    cam_fps=cam_fps, smoothing_kernel=smoothing_kernel, movement_percentile= movement_percentile)
        if output_csv and output_filepath is not None:
            if not output_filepath.endswith('.csv'):
                output_filepath += '.csv'
            try:
                state_dataframe.to_csv(output_filepath)
            except:
                traceback.print_exc()
        return state_dataframe
    
    def frame_state_df(self, state_df, output_csv=False, output_filepath= None, tolerance=None):
        """
        Create dataframe of camera times, scaled state data, and nearest frames

        Args:  
            state_df (pd.DataFrame): 

            tolerance (float ; default is None): 

        Returns:
            frame_state_df (pd.DataFrame)
        
        """
        # create a copy of state dataframe for modification
        statedf_copy = state_df.copy()

        # min max normalize pupil area and motion
        motion_scaled = MinMaxScaler().fit_transform(statedf_copy['motion_smooth'].to_numpy().reshape(-1, 1)).flatten()
        pupil_scaled = MinMaxScaler().fit_transform(statedf_copy['pupil_area'].to_numpy().reshape(-1, 1)).flatten()
        treadmill_scaled = MinMaxScaler().fit_transform(statedf_copy['treadmill_raw'].to_numpy().reshape(-1, 1)).flatten()
        motion_bool, locomotion_bool = statedf_copy['motion'], statedf_copy['locomotion']

        # drop curr cols and replace with minmax normalized 
        statedf_copy.drop(['motion_raw', 'motion_smooth', 'pupil_area', 'treadmill_raw'], axis=1, inplace=True)

        # create dataframe for used merging to state vals/cam times
        frame_df = pd.DataFrame({'nearest_frame_idx': np.arange(self.s2p_out.nframes),
                                  'frame_start_time': self.scope_times}).reset_index()
        # merge on nearest 
        nearest_frames = pd.merge_asof(statedf_copy[['time']],
                                        frame_df[['nearest_frame_idx', 'frame_start_time']],
                                          left_on='time', right_on='frame_start_time',
                                            direction='forward', tolerance=tolerance)
        frame_state_df = nearest_frames
        data_to_add = {'motion_bool': motion_bool,
                       'locomotion_bool': locomotion_bool,
                       'motion': motion_scaled,
                        'pupil':pupil_scaled,
                          'treadmill':treadmill_scaled}

        frame_state_df = pd.concat([frame_state_df, pd.DataFrame(data_to_add)], axis=1)

        frame_state_df['nearest_frame_idx'] = frame_state_df['nearest_frame_idx'].fillna(-1).astype(int)

        if output_csv:
            if not output_filepath.endswith('.csv'):
                output_filepath += '.csv'
            if output_filepath is None:
                output_filepath = os.path.join(self.data_basepath, 'frame_state_df.csv')
            frame_state_df.to_csv(output_filepath)
        return frame_state_df
    
t = TwoPData('/mnt/Gianna_Mattessich/Arenski_Data/NGF_MMN/WT/WT7628/WT7628_day1')


Reading Intan Technologies RHD Data File, Version 1.5

Found 0 amplifier channels.
Found 0 auxiliary input channels.
Found 0 supply voltage channels.
Found 8 board ADC channels.
Found 0 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

Getting transitions from 0:7691.47195 s
Found 10400 raw triggers
Could not confirm alignment of scope and triggers.
Getting transitions from 0:7691.47195 s
Found 2348 raw triggers
Getting transitions from 0:7691.47195 s
Found 229569 raw triggers
Getting transitions from 0:7691.47195 s
Found 1347915 raw triggers
Found facemap file path /mnt/Gianna_Mattessich/Arenski_Data/NGF_MMN/WT/WT7628/WT7628_day1/WT7628_10_21_24_Lamp5GCamp_V1_MMN_face_2024-10-21-155230-0000_proc.npy


In [16]:
t_statedf = t.make_state_df()
frame_state_df = t.frame_state_df(t_statedf)

Motion threshold value is: 0.2999991275671337
Proportion above threshold: 0.2312333111177903
